This sample illustrates decoding which facilitates producer-consumer pattern.

- FFMpeg is launched in a subprocess which does the following:
    - Takes the input (H.264 and HEVC).
    - Demuxes and puts Annex.B video stream into pipe.
- `PyDecoder` takes input from pipe and decodes it.
- Color conversion and JPEG encoding are done.
- Frames are shown.

In [1]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [2]:
import python_vali as vali
import numpy as np

from io import BytesIO
import subprocess

from PIL import Image
from IPython.display import display

import json

In [3]:
def get_codec_name(url: str) -> str:
    """
    This function extracts video codec name using ffprobe.

    Args:
        url (str): input file URL.

    Raises:
        ValueError: if codec or pixel format are not supported.

    Returns:
        str: codec name.
    """

    cmd = [
        "ffprobe",
        "-v",
        "quiet",
        "-print_format",
        "json",
        "-show_format",
        "-show_streams",
        url,
    ]
    ffmpeg_proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    stdout = ffmpeg_proc.communicate()[0]

    json_out = json.load(BytesIO(stdout))
    codec_name = ""

    if not "streams" in json_out:
        return codec_name

    for stream in json_out["streams"]:
        if stream["codec_type"] == "video":
            codec_name = stream["codec_name"]
            is_h264 = True if codec_name == "h264" else False
            is_hevc = True if codec_name == "hevc" else False
            if not is_h264 and not is_hevc:
                raise ValueError(
                    "Unsupported codec: "
                    + codec_name
                    + ". Only H.264 and HEVC are supported in this sample."
                )
            return codec_name
    return codec_name

In [4]:
def rtsp_client(gpu_id: int, url: str) -> None:
    """
    This function launches RTSP client which decodes video and
    presents it to user as series of decoded frames.

    Args:
        gpu_id (int): GPU ordinal.
        url (str): input file URL.

    Raises:
        StopExecution: if things go wrong.
    """
    
    # Prepare ffmpeg arguments
    codec_name = get_codec_name(url)
    bsf_name = codec_name + "_mp4toannexb,dump_extra=all"

    cmd = [
        "ffmpeg",
        "-hide_banner",
        "-loglevel",
        "fatal",
        "-i",
        url,
        "-c:v",
        "copy",
        "-bsf:v",
        bsf_name,
        "-f",
        codec_name,
        "pipe:1",
    ]

    # Run ffmpeg in subprocess and redirect it's output to pipe
    ffmpeg_proc = subprocess.Popen(
        args=cmd, stdout=subprocess.PIPE)

    # Create HW decoder class which takes input from pipe
    py_dec = vali.PyDecoder(ffmpeg_proc.stdout, {}, gpu_id)

    # GPU-accelerated converter
    pyCvt = vali.PySurfaceConverter(
        py_dec.Format, vali.PixelFormat.RGB, gpu_id=0)

    # GPU-accelerated JPEG encoder.
    # It's faster to encode Surface on GPU and show JPEG in widget.
    pyJpeg = vali.PyNvJpegEncoder(gpu_id=0)
    pyJpegEncCtx = pyJpeg.Context(100, vali.PixelFormat.RGB)

    # Allocate surface for decoder to output
    surf_src = vali.Surface.Make(
        py_dec.Format, py_dec.Width, py_dec.Height, gpu_id=0)

    # Raw Surface, converted to RGB
    surf_dst = vali.Surface.Make(
        vali.PixelFormat.RGB, py_dec.Width, py_dec.Height, gpu_id=0)

    # Main decoding loop.
    success = True
    idx = 0
    while success:
        # Decode single Surface
        success, details = py_dec.DecodeSingleSurface(surf_src)
        if not success:
            print(details)
            raise StopExecution

        # Convert to RGB
        success, details = pyCvt.Run(surf_src, surf_dst)
        if not success:
            print(details)
            raise StopExecution

        # Compress to JPEG
        buffers, details = pyJpeg.Run(pyJpegEncCtx, [surf_dst])
        if len(buffers) != 1:
            print(details)
            raise StopExecution

        # Display in notebook.
        # The same picture is shown twice for some reason.
        jpeg_bytes = BytesIO(np.ndarray.tobytes(buffers[0]))
        print(f"Frame {idx}")
        display(Image.open(jpeg_bytes), display_id="decoded_frame")
        idx += 1

    # Wait for ffmpeg process to finish
    ffmpeg_proc.wait()

In [ ]:
vali.SetFFMpegLogLevel(vali.FfmpegLogLevel.FATAL)

rtsp_client(
    gpu_id=0,
    url="https://github.com/RomanArzumanyan/VALI/raw/refs/heads/main/tests/data/test.mp4")